In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import cv2
import numpy as np
import tensorflow as tf
import random

In [2]:
# tf.config.set_visible_devices([],'GPU')
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
base_url = './crime_data/Anomaly-Videos-Part-1/'
classes = [i for i in os.listdir(base_url) if not i.startswith('.')]
print(classes)

['RoadAccidents', 'Arson', 'Shoplifting', 'Stealing', 'Burglary', 'Fighting', 'Vandalism', 'Explosion', 'Normal_Videos_event', 'Arrest', 'Abuse', 'Robbery', 'Assault', 'Shooting']


In [4]:
class_ids = {}
id = 0
for c in classes:
    class_ids[c] = id
    id += 1

print(class_ids)

{'RoadAccidents': 0, 'Arson': 1, 'Shoplifting': 2, 'Stealing': 3, 'Burglary': 4, 'Fighting': 5, 'Vandalism': 6, 'Explosion': 7, 'Normal_Videos_event': 8, 'Arrest': 9, 'Abuse': 10, 'Robbery': 11, 'Assault': 12, 'Shooting': 13}


In [5]:
train_urls = open('./crime_data/UCF_Crimes-Train-Test-Split/Action_Regnition_splits/train_001.txt','r').read().split('\n')
len(train_urls)

532

In [6]:
random.shuffle(train_urls)

In [7]:
val_urls = [ train_urls.pop(random.randint(0,len(train_urls)-1)) for _ in range(int(0.20 * len(train_urls)))]
print('train length: ',len(train_urls))
print('val length: ', len(val_urls))

train length:  426
val length:  106


In [8]:
test_urls = open('./crime_data/UCF_Crimes-Train-Test-Split/Action_Regnition_splits/test_001.txt','r').read().split('\n')
len(test_urls)

168

In [9]:
def format_frames(frame):
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.random_crop(frame, (240,280,3))
    return frame

def frames_from_video_file(video_path, n_frames):
  result = []
  src = cv2.VideoCapture(str(video_path)) 
  start = 0
  src.set(cv2.CAP_PROP_POS_FRAMES, start)

  ret, frame = src.read()
  result.append(format_frames(frame))
  total_frames = int(src.get(cv2.CAP_PROP_FRAME_COUNT))-1
  for _ in range(total_frames):
    ret, frame = src.read()
    if ret:
      frame = format_frames(frame)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  rind = []
  while(len(rind) < n_frames):
    r = random.randint(0,len(result)-1)
    if r not in rind:
      rind.append(r)
  rind.sort()

  RESULT = [result[i] for i in rind]
  
  RESULT = np.array(RESULT)[..., [2, 1, 0]]

  return RESULT

class FrameGenerator:
  def __init__(self,paths, n_frames,testing = False):
    self.n_frames = n_frames
    self.paths = paths
    self.testing = testing

  def get_files_and_class_names(self):
    video_paths = []
    labels = []
    for path in self.paths:
      video_paths.append(f'{base_url}{path}'.strip())
      labels.append(path.split('/')[0])
    pairs = list(zip(video_paths,labels))
    return pairs

  def __call__(self):
    pairs = self.get_files_and_class_names()

    if not self.testing:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames)
      label = class_ids[name]
      yield video_frames, label

In [10]:
nframes = 15
batch_size = 1
height = 240
width = 280
output_signature = (tf.TensorSpec(shape = (nframes, height, width, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(train_urls,nframes),output_signature=output_signature)
val_ds = tf.data.Dataset.from_generator(FrameGenerator(val_urls,nframes),output_signature=output_signature)
test_ds = tf.data.Dataset.from_generator(FrameGenerator(test_urls,nframes),output_signature=output_signature)
train_ds = train_ds.repeat().batch(batch_size)
val_ds = val_ds.repeat().batch(batch_size)
test_ds = test_ds.batch(batch_size)

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, TimeDistributed, Dropout, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.losses import SparseCategoricalCrossentropy

In [12]:
model = Sequential()

In [13]:
model.add(Input(shape=(nframes, height, width, 3)))
model.add(TimeDistributed(Conv2D(16,3, activation='relu')))
model.add(TimeDistributed(MaxPooling2D((4,4))))
model.add(TimeDistributed(Conv2D(32,3, activation='relu')))
model.add(TimeDistributed(MaxPooling2D((4,4))))
model.add(TimeDistributed(Conv2D(64,3, activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2,2))))
model.add(TimeDistributed(Conv2D(64,3, activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2,2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(32))
model.add(Dense(len(classes)))

In [14]:
model.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 15, 238, 278,   │           448 │
│ (TimeDistributed)               │ 16)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 15, 59, 69, 16) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 15, 57, 67, 32) │         4,640 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 15, 14, 16, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 15, 12, 14, 64) │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 15, 6, 7, 64)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 15, 4, 5, 64)   │        36,928 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 15, 2, 2, 64)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 15, 256)        │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 14)             │           462 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,966 (382.68 KB)

 Trainable params: 97,966 (382.68 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
#Some more parameters
#experiment with the batch size
#300 is too high; kernel reset
#150 working well for Conv2d+LSTM model

steps_per_epoch = 0

validation_steps = 0

num_train_sequences = len(train_urls)
num_val_sequences = len(val_urls)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
print(f'steps_per_epoch: {steps_per_epoch}')
print(f'validation_steps: {validation_steps}')

steps_per_epoch: 426
validation_steps: 106


In [16]:
earlystopping = EarlyStopping(patience=10)
reducelr = ReduceLROnPlateau(factor=0.2,patience=5)

In [17]:
history = model.fit(train_ds,steps_per_epoch=steps_per_epoch,epochs=50,callbacks=[earlystopping,reducelr], validation_data=val_ds,validation_steps=validation_steps)

Epoch 1/50
140/426 ━━━━━━━━━━━━━━━━━━━━ 44:12 9s/step - accuracy: 0.0850 - loss: 2.6338